In [1]:
# Dependencies
import numpy as np
import tweepy
import time
import json
import pandas as pd

# Twitter API Keys
# consumer_key = "TEs2QmTsz4ylhWSqXWHOvVn99"
# consumer_secret = "yA5M95uhTdeAKLondYQ9LDMPR5LWKc6wX70Yry1oRDHWtBN7gg"
# access_token = "400199431-wb53H1jfbkDwY0xVS9aZbgWZrWlXcWkhXdFuBKMh"
# access_token_secret = "IRvQVov2vmopxjeTutR0jn57lnMlYtr8syHQrGbLgbmW9"

consumer_key = "PAmpYOq5LwscMnRSFBQqwNjxH"
consumer_secret = "uvvEfLcCoU9zBsRwc3wbWNGwUy7BUVtYH3376MHpLDy7oZ6qUK"
access_token = "907734237478080523-BAtPF5z8EzzwyPef0f04JC12LOQSCim"
access_token_secret = "0F5xIDmAwVAy7pQcOWs0mMFDedr9po1tLJuLNk8r3gMuG"

# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit = True, wait_on_rate_limit_notify = True)


In [2]:
# Get a list of locations in and outside the belt (in-belt, out-belt)
base_data = pd.read_csv("Zipcode_States_With_Geo_Cordinates.csv")
lat = base_data.groupby("State")["Latitude"].mean()
lon = base_data.groupby("State")["Longitude"].mean()
dfff = {"lat":[], "lon":[]}
for each in lat.index:
    dfff["lat"].append(lat[each])
    dfff["lon"].append(lon[each])
hah = pd.DataFrame(dfff)
base_data_state = hah.set_index([lat.index])

base_data_state.head(10)

,lat,lon
State,,
Alabama,32.886361,-86.813639
Alaska,61.456423,-152.486981
Arizona,33.610082,-111.647927
Arkansas,35.124723,-92.402676
California,36.170738,-119.792701
Colorado,39.264176,-105.317544
Connecticut,41.563130,-72.804928
Delaware,39.300372,-75.526776
District of Columbia,38.896231,-77.021235


In [3]:
base_data_state = base_data_state.reset_index()
base_data_state

,State,lat,lon
0,Alabama,32.886361,-86.813639
1,Alaska,61.456423,-152.486981
2,Arizona,33.610082,-111.647927
3,Arkansas,35.124723,-92.402676
4,California,36.170738,-119.792701
5,Colorado,39.264176,-105.317544
6,Connecticut,41.563130,-72.804928
7,Delaware,39.300372,-75.526776
8,District of Columbia,38.896231,-77.021235
9,Florida,28.129716,-81.956165


In [4]:
tweet_df = {}
tweet_df = pd.DataFrame(tweet_df)
tweet_df["state"] = ""
tweet_df["text"] = ""
tweet_df["lat"] = ""
tweet_df["lon"] = ""
tweet_df["followers"] = ""
tweet_df["favorites"] = ""
tweet_df["friends"] = ""
tweet_df["statuses_count"] = ""
tweet_df["compound"] = ""
tweet_df["pos"] = ""
tweet_df["neg"] = ""


tweet_df.head()


,state,text,lat,lon,followers,favorites,friends,statuses_count,compound,pos,neg


In [5]:
# Loop through DataFrame for each Latitude and Longitude
k = 0
for index, row in base_data_state.iterrows():    
    
    # Get Tweets for all locations
    location = "%s,%s,100mi" % (row["lat"], row["lon"])     
    public_tweets = api.search("", geocode = location, count=200)
    
    # Extract all hashtags from tweets directed to the account
    for tweet in public_tweets["statuses"]:

        # Print tweet in JSON
        # print(json.dumps(tweet, sort_keys=True, indent=4, separators=(',', ': ')))

        # Print the Tweet Text         
        print(tweet["text"])

        # Print tweet text
        tweet_df.set_value(k, "state", row["State"])
        tweet_df.set_value(k, "text", tweet["text"])
        tweet_df.set_value(k, "lat", row["lat"])
        tweet_df.set_value(k, "lon", row["lon"])

        tweet_df.set_value(k, "followers", tweet["user"]["followers_count"])
        tweet_df.set_value(k, "favorites", tweet["user"]["favourites_count"])
        tweet_df.set_value(k, "friends", tweet["user"]["friends_count"])
        tweet_df.set_value(k, "statuses_count", tweet["user"]["statuses_count"])

        # Save to a csv for every state
        tweet_df.to_csv("Tweet_Dump.csv")
        
        k = k + 1
         

The last visits -----&gt; https://t.co/iVCRNsA5wR
RT @GDouglasJones: “I don’t think that’s the future,”
We agree! https://t.co/wnmVUuQ1u7
RT @YouLove_Ebony: I salute everybody thats out here getting money and bettering themselves!!👌🏾💪🏾Shit dont be easy
We just not on the same level no more and that’s fine. 🤷🏽‍♀️ everybody take different paths in life.
RT @UWA_Softball: Our own Bree Thompson singing "Land of the FREE and home of the BRAVE!" https://t.co/XKRe6dm191
I will say this. I think I have a stunningly beautiful mind
RT @CalebHorton5: I'm very blessed and I want to thank God for receiving  my 2nd offer from Lindenwood University - Belleville🏈🙏🏽# Go Lynx…
I added a video to a @YouTube playlist https://t.co/Z94SCSnUzq Anohana OP - "Aoi Shiori" | ENGLISH ver | AmaLee &amp; Dima
RT @HailStateMKTG: Celebrate #MaroonFriday tomorrow at @HailStateSOC at 7PM!

The first 250 fans at the MSU Soccer Field can get this FREE…
Halftime: Lee 29, Carver 20.
RT @occdissent: I'm not interested in op

食べたらぁ！！！！
今日の診察で体重のこと言われなかったら、ドーナツ2個食べていいこととする
"LUST" Offical Music Video Out Now⚰🌹
LINK IN BIO. https://t.co/T2S4jmLDV2
"LUST" Offical Music Video Out Now⚰🌹 
LINK IN BIO. https://t.co/0N4WE8WCag
【微小地震速報】
2017/09/29 3:34:03 JST, 
アメリカ合衆国 アンカレッジの西南西95km, 
ML2.1, TNT21.3kg, 深さ73.2km, 
https://t.co/YTFQr85jGs
2.1 magnitude #earthquake. 38 km from #Nikiski, AK, United States https://t.co/76IRzidmPS
USGS reports a M2.1 #earthquake 37km NNW of Nikiski, Alaska on 9/28/17 @ 18:34:03 UTC https://t.co/83QZnLSNkJ #quake
สั่งไปแล้ววว 1030บาทฮืออ
อยู่บ้านคนเดียว น่ากลัวเว้ย
【微小地震速報】
2017/09/28 20:34:50 JST, 
アメリカ合衆国 アンカレッジの北西132km, 
ML2.2, TNT30.1kg, 深さ87.1km, 
https://t.co/mISmcpquoS
2.1 magnitude #earthquake. 70km WSW of Talkeetna, Alaska https://t.co/3JYHQsRlZn
USGS reports a M2.1 #earthquake 70km WSW of Talkeetna, Alaska on 9/28/17 @ 11:34:50 UTC https://t.co/smrGtRm5sS #quake
ヴ20話は、ビットロンとデジトロンが召喚されたときが一番テンション上がった
ไม่เคยคิดอ่ะ
มาร้องเพลงชาติล่ะก็กลับอืมมมมม555
when you lack communication you se

@TimberPuppers Also, i sent this to the wrong account again at first. :/
RT @RepRubenGallego: This is outrageous. Trump administration officials are spending more $ on private jets than many Americans make in a y…
@brunomccam @ClaromusicaAR JAJAJAJJAJA
👀👀👀👀👀 https://t.co/gKzQJjtMMy
Check it out! I just voted for ASU winning the 1987 Rose Bowl in the #RoseBowl1987 Tug o' War… https://t.co/n6qr7sWFCj
RT @bannerite: GOP elected a birther Potus so why not a truther for AL Senator? They will take anything to keep power. https://t.co/Yr4jZk8…
RT @cubeunited: 2017 AAA 팬투표 인기상 가수 부문 투표가 진행되고 있습니다.
#아잉 #멜로디 #유니버스 여러분의 많은 참여 부탁드립니다 🙌🏻 https://t.co/79q5ATG2um
@NFL Lighting from the powerful God🙏
RT @kelliwardaz: Wow. $135,000,000,000. Every. Single. Year. Unbelievable. #MixTheMortarToFixTheBorder #StopIllegalImmigration https://t.co…
https://t.co/L5oFMlRHiy
@thehill Once again @realDonaldTrump more worried about meaningless BS than actual American citizens. He is a compl… https://t.co/qp9SRUo8TO


RT @damnitsanthony: Hint hint wink wink bitchhh fucking dm me pendja!! https://t.co/9uEz87mXBD
I'm so fucking frustrated
RT @Adbrown77a: @Thomas1774Paine Too late ... damage has already been done.  Football (that I've faithfully watched my entire life) is over…
@SBaabeee @Shyboy563 Which one is me?
A2: Ss might not think that D and B are the same shape. #elemmathchat
Romo so enthusiastic about this rain in his cozy booth...
@MattBarton2 @sfmjm @MikeSilver No you didn't
RT @lucyhale: @SashaaPieterse @GMA Beautiful girl!!!!!!!!
RT @damnitsanthony: Hint hint wink wink bitchhh fucking dm me pendja!! https://t.co/9uEz87mXBD
RT @damnitsanthony: Hint hint wink wink bitchhh fucking dm me pendja!! https://t.co/9uEz87mXBD
RT @MissConnieTran: #BREAKING: Another rock fall near #ElCapitan in @YosemiteNPS. My photographer and I witnessed the whole ordeal, it was…
RT @Curlyymar_: 😭😝😝😝 https://t.co/GJPx35eHGP
Who wants to go on a walk wiff me.
Hot! iPhone 8 Plus Case, iPhone 7 Plus Case, Slim Shock-ab

@ShaunKing @FBI Outstanding!
This sweater has the most darling detail on the back! And have I mentioned how much I love this bag? 🖤… https://t.co/OqwNFNLkgo
RT @Broncos: "You just have to get him on the ground." - @VonMiller on slowing down @MoneyLynch

#BroncosIn60
https://t.co/wKQfKzfB53
RT @TamaraMcCleary: "If you're lucky enough to do well, it's your responsibility to send the elevator back down." #quote #leadership #inspi…
RT @nicholaspardon: This is why I drive a @Tesla. https://t.co/8U4DuD2xfL
Eyebrows waxed as well cause this growing them out shit ain't working
RT @GeorgeBrauchler: @davidsirota A progressive who sees and pushes race as the motivation for everything. Pretty obvious intellectual corr…
Introverts speaking well? Yes, you can make it happen: https://t.co/TrLwmuBkza #engchat #langchat
Saucin saucin on youuuuuuu
@NoSlackDelta @NHL https://t.co/V5X4rqE5ui
RT @PrincessMarxxx: The #espresso is ALWAYS #amazing at @EspressoCielo, but today, the beans are in a sweet spot --

...

@adult_mom @PetrickSara Then you need two more. Don't give up.
RT @KattyKayBBC: PR. It's been 8 days, 40% don't have clean drinking water, containers of supplies sit in port. Nothing unfair about that.…
RT @Redskins: Why we love sports today.

👏👏 @Fatrob32 &amp; @Truthh4: https://t.co/TwlXQ3lQPj https://t.co/MDqpjO77Rz
RT @RealMattCouch: I Dare you to RT This! Joe Capone, Mayor Bowser, Liz Lyons, all visited the White House within a week of Seth Rich's Mur…
RT @JoeBiden: We Veeps stick together. Jill and I, and all of the Bidens, are with you, Julia. https://t.co/JP0c2wtrJ6
RT @politico: Christopher Wray was installed as the eighth FBI director today. Notable absences: Predecessors Comey and Mueller https://t.c…
Are your #S3 Buckets #Secure? @Evident.io 
https://t.co/1AIV7Z9CtO https://t.co/p6xKweQaF2
RT @mattklewis: Re-upping this from @thedailybeast  https://t.co/mxUDiSrai7
RT @realDonaldTrump: FEMA &amp; First Responders are doing a GREAT job in Puerto Rico. Massive food &amp; water

RT @MarinersStore: Alright, who's feeling lucky?

Follow &amp; RT for a chance to win a Levi's jean Mariners jacket. #TrueToTheBlue https://t.c…
@Inked1BNA We want to keep you on the Magenta side, Kevin. Send us a DM so we can get more on your feedback &amp; check… https://t.co/NYbJrTMKom
Happy birthday @jakeseidel11 https://t.co/zQwc8PnSJa
RT @OriginalFunko: RT &amp; follow @OriginalFunko for the chance to WIN a #Thorsday Prize Pack! #ThorRagnarok https://t.co/xD553bWHZ3
I really just be watching old school bball games on youtube
@NoSpandexReq @BHSafeStreets Ok
RT @OriginalFunko: RT &amp; follow @OriginalFunko for the chance to win a @Walmart exclusive #ThorRagnarok Hulk Pop! https://t.co/DUI5fvLcKK
Trying Julia.
RT @Seahawks: #Seahawks 
#Colts 

#TBT https://t.co/9seOnq2d0S
RT @MarinersStore: Alright, who's feeling lucky?

Follow &amp; RT for a chance to win a Levi's jean Mariners jacket. #TrueToTheBlue https://t.c…
RT @toojazzy25: 방탄이 뭔가 심상치 않다는걸 느끼고 처음 이야기했던게 2014년 케이콘을 취재한 직후였는데, 

Did you get your Hazmat nuclear ready https://t.co/bwSNPEGntR
true friendship is calling your roommate and pretending to cry bc she needs to get out of a bad date
@Softykjr Love my HAWKS from 1976 till now always a fan I can separate football from off the field issues
RT @gaylanotkayla: @Dr_SofaKingSXE Dreamboy
RT @acamire: Praise God till your last breath.
RT @kourtrob: @ gig harbor 
take notes: 
bellingham plants trees instead of cutting them down
not that long ago but feels like another life
2 more daysssss https://t.co/hVWrAd4mHL
Bae is killing zombies for me https://t.co/Trn1oDQajS
@adamrank Smallwood or Lamar miller Sunday?
@Theresacaputo I love you Theresa! 😊💕🙈
☹️  RT @RockRidgeFB: Final. Tuscarora 42 - Rock Ridge 6.
is my layout okay or
Retweeted Jack Posobiec (@JackPosobiec):

BOOM: Trump Preparing New Executive Order to Let Americans Purchase... https://t.co/eaXAoiFcYn
RT @BraunDuinIt: WOTC finally released the art for the Charging Monstrosaur Game Day Promo. https://t.co/cJt

RT @packers: ⛈⛈⛈

#CHIvsGB https://t.co/Wk3g4cRP8y
RT @380kmh: while u begged for treats I studied the blade https://t.co/K5zY792AKy
The field and stands are soaked. https://t.co/VXxgnYaDMh
@nickjonas @cbear213 @Elle_Benson
RT @packers: ⛈⛈⛈

#CHIvsGB https://t.co/Wk3g4cRP8y
The Woodmans playlist has gone from Avril Lavigne, to Sheryl Crow, to Clay Aiken and I am LIVING
RT @ChadHarnisch: A3 I spend a lot of time pursuing DuFour's definition of consensus...will of the group is  known, all dissent has been he…
RT @packers: Here's @ClayMatthews52's record-setting SACK! 

#CHIvsGB #GoPackGo https://t.co/TqDYgnXrZQ
In other news, the Brewers won their final home game of the year!
RT @packers: ⛈⛈⛈

#CHIvsGB https://t.co/Wk3g4cRP8y
RT @380kmh: while u begged for treats I studied the blade https://t.co/K5zY792AKy
RT @packers: ⛈⛈⛈

#CHIvsGB https://t.co/Wk3g4cRP8y
You just gotta love it when Mich McConnell and his ilk come out looking like the douche-bags that they are... and w… https://t.co/iYe

You can know who has entered your profile here https://t.co/QNVOUWOsD3
RT @peak_wrestling: Can't wait for these studs and the rest of our team to tear it up this season! @That_dude_jp @javenpalmer @nathancaleb1…
@monicaraymund @lizajbennett mi Corazon se está 💔 por todos que no saben sobre sus familias 😭 #PuertoRicoselevanta
Swapping incoherent action sequences for incoherent dialogue now. My agent said it would be good for my bank account, that's why. 38 #LOEG
THOR RAGNORAK!  I can't freakin' wait!  #GreysAnatomy #TGIT
RT @19Pam56: No, no, no!  Much easier just to tweet based on headline!  See:Valerie Plame. https://t.co/bNQDIrQ8lS
@beautyyfulsoul I'm wondering if they're getting it all out in the first episode and then they'll scale it back
@TwitchSupport I am trying to finish up the Affiliate tax on my dashboard, But it seems to be stuck on Payment Method under review for 7 hrs
The first Nutso day is this Saturday!!! Meet us there! @nutmgg https://t.co/rw81prnpRM
https://t.co/0hu3hv

In [18]:
# We're going to store the tweets inside a dataframe associated with the state, location, and in-belt/out-belt 

In [19]:
# We're going to process all the tweets

In [20]:
# We're going to group the data in-belt/out-belt